In [1]:
from tqdm import tqdm
import matplotlib.pyplot as plt # type: module
import matplotlib.ticker as ticker
from matplotlib import colormaps
from matplotlib.colors import Normalize

import numpy as np
import os, glob
import time
import warnings

from rur.fortranfile import FortranFile
from rur import uri, uhmi, painter, drawer
from rur.sci.photometry import measure_luminosity
from rur.sci.geometry import get_angles, euler_angle
from rur.utool import rotate_data
from scipy.ndimage import gaussian_filter
uri.timer.verbose=1
# from rur.sci.kinematics import f_getpot

from icl_IO import mode2repo, pklsave, pklload
from icl_tool import *
from icl_numba import large_isin, large_isind, isin
from icl_draw import drawsnap, add_scalebar, addtext, MakeSub_nolabel, label_to_in, fancy_axis, circle
import argparse, subprocess
from importlib import reload
import cmasher as cmr
from copy import deepcopy

In [2]:
mode = 'nh'
iout = 1026
repo, rurmode, dp = mode2repo(mode)
snap = uri.RamsesSnapshot(repo, iout, mode=rurmode)
snaps = uri.TimeSeries(snap)
snaps.read_iout_avail()
nout = snaps.iout_avail['iout']
gals = uhmi.HaloMaker.load(snap, galaxy=True, double_precision=dp)
hals = uhmi.HaloMaker.load(snap, galaxy=False, double_precision=dp)

LG = pklload("./database/00_LocalGroup_fix.pickle")
print(LG.keys())
print(LG[2].keys())

[Output 01026] Age (Gyr) : 11.624 / 13.741, z = 0.17149 (a = 0.8536)
dict_keys([1, 2, 3, 4, 7, 8, 9, 10, 14, 17, 23, 24, 25, 26, 29, 35, 36, 37, 40, 46, 47, 52, 63, 64, 75, 84, 92, 107, 108, 121, 129, 207, 11])
dict_keys(['BGG', 'neighbor', 'sats', 'subs', 'real'])


See `05a_measure_subprop.py`  
Output:
- `./database/parts/insub/nh_star_{key:04d}_{sub['id']:07d}.pickle`
- `./database/parts/insub/nh_cell_{key:04d}_{sub['id']:07d}.pickle`
- `./database/parts/insub/nh_dm_{key:04d}_{sub['id']:07d}.pickle`

In [3]:
key = 1
LG[1]['subs']['id']

array([ 1310,  1324,  1940,  1941,  3484,  3797,  3802,  3885,  5008,
       20689, 20699, 20700, 20705, 20706, 20708, 20709, 20710, 20713,
       20714, 20716, 20717, 20718, 20720, 20721, 20722, 20723, 20724,
       20726, 20727, 20730, 20731, 20732, 20733, 20734, 20735, 20736,
       20737, 20738, 20739, 20742, 20743, 20744, 20746, 20747, 20748,
       20749, 20750, 20751, 20753, 20754, 20756, 20757, 20758, 20760,
       20762, 20763, 20764, 20766, 20767, 20771, 20774, 20776, 20778,
       20779, 20780, 20781, 20783, 20784, 20785, 20787, 20790, 20792,
       20794, 20795, 20798, 20799, 20800, 20803, 20804, 20805, 20806,
       20809, 20810, 20813, 20814, 20815, 20817, 20820, 20823, 20824,
       20826, 20827, 20829, 20830, 20831, 20833, 20836, 20840, 20842,
       20844], dtype=int32)